# SageMaker Pipeline (pipeline.py) 로컬에서 Test 하기



## 중요 가이드
- 디렉토리 설명
- 추가 Dependency 폴더 (예: 소스 폴더) 필요시 설정 방법
- Test 폴더
- get_pipeline() 를 통한 파이프라인 실행

```pythona
from pipelines.abalone.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)
pipeline.upsert(role_arn=role)
execution = pipeline.start()
```
- 파라미터 제공 통한 실행

```python
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)
```

# 1. 환경 설정 및 컨피그 파일 로딩

In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
code_pipeline_serving_config_json_path = 'pipelines/ncf/src/code_pipeline_serving_config.json'
sm_pipeline_serving_config_json_path = 'pipelines/ncf/src/sm_pipeline_serving_config.json'

In [3]:
from pipelines.ncf.src.common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))


Code Pipeline Series Params: 
{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/codebuild-gsmoon",
  "project_prefix": "CodePipeline-Serving-NCF",
  "region": "us-east-1",
  "account_id": "057716757052",
  "serving_code_repo_name": "ncf-serving",
  "code_build_project_name": "ncf-serving-sm-pipeline",
  "bucket": "sagemaker-us-east-1-057716757052",
  "code_pipeline_name": "ncf-serving-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "endpoint_name": "ncf-codepipeline-endpoint",
  "branch_name": "master"
}
SageMaker Pipeline Series Params: 
{
  "sm_pipeline_name": "sm-serving-pipeline",
  "endpoint_instance_type": "ml.g4dn.xlarge",
  "ModelApprovalStatus": "Approved",
  "bucket": "sagemaker-us-east-1-057716757052"
}


### 필요한 설정 값 로딩

In [11]:
import boto3
import sagemaker
import os

region = code_pipeline_serving_dict["region"]
bucket = code_pipeline_serving_dict["bucket"]
role = code_pipeline_serving_dict["code_build_service_arn"]
model_package_group_name = code_pipeline_serving_dict["model_package_group_name"]
project_prefix = code_pipeline_serving_dict["project_prefix"]
endpoint_name = code_pipeline_serving_dict["endpoint_name"]

pipeline_name = sm_pipeline_serving_dict["sm_pipeline_name"]
endpoint_instance_type = sm_pipeline_serving_dict["endpoint_instance_type"]
ModelApprovalStatus = sm_pipeline_serving_dict["ModelApprovalStatus"]


In [12]:
print("project_prefix: \n", project_prefix)
print("region: \n", region)
print("role: \n", role)
print("bucket: \n", bucket)
print("model_package_group_name: \n", model_package_group_name)
print("endpoint_name: \n", endpoint_name)

print("ModelApprovalStatus: \n", ModelApprovalStatus)
print("pipeline_name: \n", pipeline_name)
print("endpoint_instance_type: \n", endpoint_instance_type)

project_prefix: 
 CodePipeline-Serving-NCF
region: 
 us-east-1
role: 
 arn:aws:iam::057716757052:role/codebuild-gsmoon
bucket: 
 sagemaker-us-east-1-057716757052
model_package_group_name: 
 NCF-Model-CodePipeline
endpoint_name: 
 ncf-codepipeline-endpoint
ModelApprovalStatus: 
 Approved
pipeline_name: 
 sm-serving-pipeline
endpoint_instance_type: 
 ml.g4dn.xlarge


# 3. Pipeline 테스트

In [25]:
from pipelines.ncf.pipeline import get_pipeline


pipeline = get_pipeline(
    project_prefix = project_prefix,
    region = region,
    endpoint_name = endpoint_name, # Sagemaker Endpoint Name
    role= role, # SAGEMAKER_PIPELINE_ROLE_ARN 이 넘어옴.
    default_bucket=bucket,
    model_package_group_name= model_package_group_name,
    pipeline_name= pipeline_name
)

######### get_pipeline() input parameter ###############
### BASE_DIR: /home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/5_sm-serving-codepipeline/codecommit/pipelines/ncf
project_prefix: CodePipeline-Serving-NCF
role: arn:aws:iam::057716757052:role/codebuild-gsmoon
default_bucket: sagemaker-us-east-1-057716757052
model_package_group_name: NCF-Model-CodePipeline
endpoint_name: ncf-codepipeline-endpoint
pipeline_name: sm-serving-pipeline
######### Look at subfolder and files #########
./img
    ./img/pipeline-full.png
./tox.ini
./codebuild-buildspec.yml
./tests
    ./tests/test_pipelines.py
    ./tests/.ipynb_checkpoints
        ./tests/.ipynb_checkpoints/test_pipelines-checkpoint.py
./LICENSE
./pipelines
    ./pipelines/_utils.py
    ./pipelines/.ipynb_checkpoints
        ./pipelines/.ipynb_checkpoints/run_pipeline-checkpoint.py
    ./pipelines/__version__.py
    ./pipelines/run_pipeline.py
    ./pipelines/__pycache__
        ./pipelines/__pycache__/__init__.

In [26]:
definition = json.loads(pipeline.definition())
# definition

In [27]:
pipeline.upsert(role_arn=role)
#execution = pipeline.start()
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus = ModelApprovalStatus,                                
        endpoint_instance_type = endpoint_instance_type
    )
)

In [28]:
execution.wait()
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/sm-serving-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/sm-serving-pipeline/execution/kwug226kcb9d',
 'PipelineExecutionDisplayName': 'execution-1668941150872',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'sm-serving-pipeline',
  'TrialName': 'kwug226kcb9d'},
 'CreationTime': datetime.datetime(2022, 11, 20, 10, 45, 50, 771000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 11, 20, 10, 50, 42, 769000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '07abca57-121e-41a7-b45f-6165b4c80b14',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '07abca57-121e-41a7-b45f-6165b4c80b14',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '498',
   'date': 'Sun, 20 Nov 2022 10:50:52 GMT'},
  'RetryAttempts': 0}}

In [29]:
execution.list_steps()

[{'StepName': 'LambdaDeployStep',
  'StartTime': datetime.datetime(2022, 11, 20, 10, 45, 57, 222000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 20, 10, 50, 42, 311000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Lambda': {'Arn': 'arn:aws:lambda:us-east-1:057716757052:function:sagemaker-lambda-step-endpoint-deployment',
    'OutputParameters': [{'Name': 'other_key',
      'Value': 'ncf-codepipeline-endpoint'},
     {'Name': 'body', 'Value': '"Created Endpoint!"'},
     {'Name': 'statusCode', 'Value': '200.0'}]}}},
 {'StepName': 'MyModelCreationStep-CreateModel',
  'StartTime': datetime.datetime(2022, 11, 20, 10, 45, 55, 583000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 20, 10, 45, 56, 668000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model/pipelines-kwug226kcb9d-mymodelcreationstep--sdl9ozafsr'}}},
 {'StepName': 